# Final Project of Applied Data Science Capstone
# - Find a new living area in Toronto

## 1. Introduction
As a real estate agent, we have a new client, David, who just got a new job in Toronto and will have to move there within 2 months. Therefore, he contracted us with this taks to find an ideal place and an apartment for him.

David is a software engineer. He works mostly from home and only needs to go to the office for meetings sometimes. Therefore, the distance between his home and office is not a problem. What really matters is the the quality of life he will have in the new neighborhood. 

David likes to taste coffee and he is also a workout fan since college. In addition, he doesn't like to cook because he hates the cooking smell inside the house, so it would be the best if there are tons of restaurants right down the street. However, David is allergic to many things so it would be better to avoid living in an area that has too many parks or natural environment.



## 2. Data for this project
To help David find the ideal place to live, we will need the facilities data from Toronto, which we will get via Foursquare. After retrieving the data, we will perform a clustering process to filter the top 3 areas for David to choose and that will be the end results of this project.